<h1 style = "color: black; background-color: yellow;" align="center"> Content Based Movie Recommendation System</h1>

* <p style = "color: white; background-color: black;font-size:14px ;"> A recommendation system is a type of information filtering system which attempts to predict the preferences of a user, 
and make suggests based on their preferences.</p> 


* <p style = "color: white; background-color: black;font-size:14px ;">  Given the huge amount of movies  are  available all over the world, it is challenging for a user to find the appropriate movies suitable for his/her tastes. 
Different users like different movies or actors. It is important to find a method of filtering irrelevant movies and/or find a set of relevant movies.<br>Movie recommendation system is a process of exactly doing  above tasks.


* <p style = "color: white; background-color: black;font-size:14px ;">This recommender system has  a very high business value as an example Netflix estimated  that Recommendation is worth a yearly $1billion. <p>
 

In [1]:
# Importing the dependencies

import pandas as pd
import difflib

from  sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.metrics.pairwise import  cosine_similarity


### Data collection and preprocessing



In [2]:
movies_data = pd.read_csv('movies.csv')


In [3]:
# Printing the 5 first rows of the dataframe 

movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.shape

(4803, 24)

In [5]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
# Selecting relevant features for recommendation
# Or feature Selection

selected_features = ['genres', 'keywords','tagline','cast','director', 'original_language' ]
print(selected_features) 

['genres', 'keywords', 'tagline', 'cast', 'director', 'original_language']


### Textual data contains a lot of missing values called as null values and we need to replace them with null string 

In [7]:
# Replacin null values with null string 

for feature in selected_features:
    
    movies_data[feature] = movies_data[feature].fillna('')

In [8]:
# Combining the all 5 selected features

combined_features = movies_data['genres']+' '+ movies_data['keywords']+' '+ movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']



In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


###  Now we are going to convert this textual data to feature vectors/ numerical values

- The reason to convert textual data into numerical values is that we cannot find the cosine similarity easily with the text data 
But if we have a numerical data then we can easily find the cosine similarity 

In [10]:
vectorizer = TfidfVectorizer()


In [11]:
# We  use vectorizer to fit and transform the data 
# So in feature_vectorizer varible I am going to all the numerical values 
# we tansform the combined_features data 

feature_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(feature_vectors[:1])

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  (0, 13599)	0.1036413987316636
  (0, 5274)	0.11108562744414445
  (0, 274)	0.09021200873707368
  (0, 201)	0.07860022416510505


### We dont feed the textual data directly to our machine so first we convert them to feature vectors and then we will feed the data to our machine learning model

<h1 style = "color: black; background-color: yellow; font-family: Comic Sans MS;" align="center">  Cosine Similarity </h1>
 

Getting the similarity scores using cosine similarity 

First we converted the textual data to numerical values and now we can feed the cosine similarity.

Cosine similarity function will go through all the numerical values and will try to find which values are similar to each other 
and the end result is that it can find which movies are related to each other.  

In [15]:
# Getting the similarity scores using cosine similarity 

similarity = cosine_similarity(feature_vectors)

In [16]:
# Now we check the similarity value

print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [17]:
print(similarity.shape)

(4803, 4803)


### (4803, 4803 )

First 4803 is the index and second 4803 represent similarity score 

It means that for each movie we will compare with all the other movies, for exp let's take ' Avatar ' movie 
Avatar movie will be compared with all the other movies in the data set and will try to give a similarity score value and then 
This will be carried out for all the movies as well 


### Now that we have the similarity score 

We can ask the user to give his/her favorite movie name, after we get to know the movie name we will try to find whether that 
Movie is present in our data set or no 
If it's present in our data set we will try to find which movies are similar to it using similarity score 

## Getting the movie name from the user 


In [18]:
movie_name = input(' Enter your favorite movie name please: ')

 Enter your favorite movie name please: Batman


###  Now 

We have to create a list which contains all the names of the movies, and the reason is that I am gonna compare it with the 
Value given by the user 

In [19]:
# Creating a list with all the movie names given in the data set

list_of_all_titles = movies_data['title'].tolist() 
print(list_of_all_titles[:4])

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises']


### Above list contains the names of all the movies that is present in our data set 

The reason we are doing is that we want to find the best match for the value given by the user so we need to chech whether this 
value is present in this particular data set or no 

So what I will do ?

I will compare 'Iron man' movie name individually with all the values in the list  

In [20]:
# Finding the close match for the movie name given by the user 

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

print(find_close_match)

['Batman', 'Batman', 'Catwoman']


#### We got 3 close matches for given movie name ' Batman '

## Now what we are going to do is :

To take the first value along because we just want one movie now and after that we will try to compare which movies are similar
To this Batman movie 
We take the first value because it's the most relevant or most similar movie name 

In [21]:
close_match = find_close_match[0]

print(close_match)

Batman


### Now 

WE want to find the index of movie

we got the close match ' BatMan ' now I am going to take this ' Batman '  and check which Particular row is this 
' Batman '

So I compare it with title column and Can find it's index through title column easily  

As the output will be in the form of list so we put values[ 0 ] to get the actual index

In [22]:
# Finding the index of the movie by using it's title 

index_of_the_movie = movies_data[ movies_data.title == close_match ]['index'].values[0]
print(index_of_the_movie)

1359


### Now

I take this index value and  find the similar movies so we can do that because we have the similarity score

The similarity score will contain the index and its a similarity score value

What we are doing here is that I take this ' Batman ' movie and I am going to find the similarity score of all the movies in 
My data set , so the movies which are similar to this 'Batman' , will have a higher similarity score and the movies which 
Are not relevant to the ' Batman  ' movie will have very small similarity score  

enumerate is used to run a loop in a particular list 

So the loop or iteration that we are doing here is we want to find the similarity score for all the movies and we don't want
To do that individually 

So enumerate basically used to carry out loop in a list and also we want to get the index on counting the loop

In [23]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie])) 
print(similarity_score[:5])

[(0, 0.02531512269737111), (1, 0.04983293064399152), (2, 0.013599520029326722), (3, 0.20438773732168222), (4, 0.024929726723526918)]


### Above

(0, 0.02531512269737111) The first value which is zero represents the index of the movie and 0.02531512269737111 represents
The similarity score of the movie compared to the 'Batman'  movie

As in the first indec the movie name is ' Avatar ' so we can see that the similarity score of Avatar movie with BatMan movie 
is very less 0.025 

In [24]:
len(similarity_score)

4803

### Once 

Once we find those values which have higher similarity score then we can recommend those movies to the user
So that will be the last step of our code

## Order

Now the similarity score is in the order of our data set I mean in the order of index 

Now we have to sort it based on the higher similarity score value to the lower similarity score value

So that we can recommend the movies with higher similarity values to the user 

AS we sort similarity score from higher to lower so we take 

    ' reverse = True '
    
For descending order 

    key = lambda x : x [ 1 ]
    
x  =  list ' Similarity score list '


    (0, 0.033570748780675445)

      
     x[ 0 ]  =  0                        -->   Index    
     x[ 1 ]  =  0.033570748780675445     -->  Similarity Score
     
     
     
     
     

In [25]:
# Sorting the movies based on their similarity score 

sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)

print(sorted_similar_movies[:10])

[(1359, 1.0), (428, 0.4311643836232694), (210, 0.25737999820859625), (3, 0.20438773732168222), (119, 0.19262528757150407), (65, 0.1775581506611392), (1512, 0.14705162654306442), (813, 0.14414128303962467), (2530, 0.13737322473729185), (1017, 0.13713281929528845)]


### Above

Above you can see that similarity score starts from a high score 

(1359, 1.0),  in the 1359th index there's a movie which have very high similarity score which is 1.0

SO the top movies in the top movies in sorted_similar_movies list have high similarity score value 

So now we can take the fisrt 10 or 20 movies from this sorted_similar_movies list and recommend to the user

### Now

I take values Individually, So I take the first element (1359, 1.0) and take the index 1359 , using the index I will find the movie from my data set 
So this will be the first step of for loop.

Loop explaination:

    for movieee in sorted_similar_movies:
    
When we run the loop for first time the movieee will take this (1359, 1.0) particular value, So in this first
Element I don't want (1359, 1.0) these two values (index, similarity score ).

I just want index which is 1359 in here and through index I can find the title of the movie 

So for this to get index we write this line :
    
    index = movie[0]
    
    movie[ 0 ] = 1359
    movie[ 1 ] = 1.0 
    
    
     

In [26]:
# As an example we will use this in for loop which is coming  next
# Do not write this in your project notebook

TitleFromIndex = movies_data[movies_data.index == 1359]['title'].values[0]
print(TitleFromIndex)

Batman


In [27]:
# Print the name of similar movies based on the index 
# Now based on the index in the sorted_similar_movies list we will find the name of the movie 

print(' Moveis suggested for you : \n')

# We take a for loop

i = 1

for movie in sorted_similar_movies:
    
    index = movie[0]
    title_from_index =  movies_data[ movies_data.index == index ]['title'].values[0]
    
# I am going to suggest 30 movies name to the user based on his/her favorite movie 

    if (i <= 30):

# I print movies with the serial number        
       
        print(i, '.', title_from_index )
        i+= 1
        
            

 Moveis suggested for you : 

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . The Dark Knight Rises
5 . Batman Begins
6 . The Dark Knight
7 . A History of Violence
8 . Superman
9 . Beetlejuice
10 . Bedazzled
11 . Mars Attacks!
12 . The Sentinel
13 . Planet of the Apes
14 . Man of Steel
15 . Suicide Squad
16 . The Mask
17 . Salton Sea
18 . Spider-Man 3
19 . The Postman Always Rings Twice
20 . Hang 'em High
21 . Spider-Man 2
22 . Dungeons & Dragons: Wrath of the Dragon God
23 . Superman Returns
24 . Jonah Hex
25 . Exorcist II: The Heretic
26 . Superman II
27 . Green Lantern
28 . Superman III
29 . Something's Gotta Give
30 . Reds


### Well this is how our recommendation system works 

Now just as a last thing I want to incorporate all the codes in a single cell

# Movie Recommendation System

In [29]:

movie_name = input(' Enter your favorite movie name please: ')

list_of_all_titles = movies_data['title'].tolist() 


find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]


index_of_the_movie = movies_data[ movies_data.title == close_match ]['index'].values[0]


similarity_score = list(enumerate(similarity[index_of_the_movie]))



sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)


print('\n\n Moveis suggested for you : \n')


i = 1

for movie in sorted_similar_movies:
    
    index = movie[0]
    title_from_index =  movies_data[ movies_data.index == index ]['title'].values[0]
     

    if (i <= 11):
       
        print(i, '.', title_from_index )
        i+= 1


 Enter your favorite movie name please: Batman


 Moveis suggested for you : 

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . The Dark Knight Rises
5 . Batman Begins
6 . The Dark Knight
7 . A History of Violence
8 . Superman
9 . Beetlejuice
10 . Bedazzled
11 . Mars Attacks!


In [31]:
def movie():
    
    feature_choice = input('Welcome to movie recommendation system\nChooose whether you want recommendation based on: movie_name | director \n ')
    
    if feature_choice == 'movie_name':
    
        movie_name = input('\nEnter your favorite movie name please: ')
        list_of_all_titles = movies_data['title'].tolist()

        if movie_name in list_of_all_titles:

            find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
            close_match = find_close_match[0]
            index_of_the_movie = movies_data[ movies_data.title == close_match ]['index'].values[0]
            similarity_score = list(enumerate(similarity[index_of_the_movie]))
            sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)
            
            print('\n.... Moveis suggested for you...\n')
            i = 1
            for movie in sorted_similar_movies:

                index = movie[0]
                title_from_index =  movies_data[ movies_data.index == index ]['title'].values[0]

                if (i <= 10):
                    print(i, '.', title_from_index )
                    i+= 1
                    
        else:
            
            print('Not Exist')
        

    elif feature_choice == 'director':
        
        director_name = input('\nEnter your favorite director name please: ')
        list_of_all_directors = movies_data['director'].tolist()
        
        if director_name in list_of_all_directors:
            
            find_close_match = difflib.get_close_matches(director_name, list_of_all_directors)
            close_match = find_close_match[0]
            
            index_of_the_director = movies_data[ movies_data.director == close_match ]['index'].values[0]
            similarity_score = list(enumerate(similarity[index_of_the_director]))
            
            sorted_similar_directors = sorted(similarity_score, key = lambda x : x[1], reverse = True)
             
            print('\n      ... Moveis suggested for you...  \n')
            i = 1
            for director in sorted_similar_directors:

                index = director[0]
                title_from_index =  movies_data[ movies_data.index == index ]['title'].values[0]

                if (i <= 10):
                    print(i, '.', title_from_index )
                    i+= 1
            
            
        else:
            
            print('Not Exist')
            
        
movie()

Welcome to movie recommendation system
Chooose whether you want recommendation based on: movie_name | director 
 movie_name

Enter your favorite movie name please: Avatar

.... Moveis suggested for you...

1 . Avatar
2 . Alien
3 . Aliens
4 . Guardians of the Galaxy
5 . Star Trek Beyond
6 . Star Trek Into Darkness
7 . Galaxy Quest
8 . Alien³
9 . Cargo
10 . Trekkies
